In [0]:
from pyspark.sql import functions as F

try:
    # 1. Obter o intervalo de datas (min e max) da tabela de pedidos
    datas_pedidos = spark.table("medalhao.silver.ft_pedidos").select(
        F.min("pedido_compra_timestamp").alias("data_min"),
        F.max("pedido_compra_timestamp").alias("data_max")
    ).first()
    
    data_min = datas_pedidos["data_min"].date()
    data_max = datas_pedidos["data_max"].date()

    # 2. Gerar o DataFrame de calendário usando 'sequence' e 'explode'
    df_calendario = spark.sql(f"SELECT explode(sequence(to_date('{data_min}'), to_date('{data_max}'), interval 1 day)) AS sk_tempo")

    # 3. Enriquecer o calendário com as dimensões de tempo
    df_dim_tempo = df_calendario.select(
        F.col("sk_tempo"),
        F.year("sk_tempo").alias("ano"),
        F.quarter("sk_tempo").alias("trimestre"),
        F.month("sk_tempo").alias("mes"),
        F.weekofyear("sk_tempo").alias("semana_do_ano"),
        F.dayofmonth("sk_tempo").alias("dia"),
        F.dayofweek("sk_tempo").alias("dia_da_semana_num"), # 1=Domingo, 7=Sábado
        
        # O Spark não traduz date_format para português, então pegamos em inglês
        F.date_format("sk_tempo", "E").alias("dia_da_semana_nome_en"), # (ex: Mon)
        F.date_format("sk_tempo", "MMMM").alias("mes_nome_en") # (ex: January)
    )

    # 4. Mapear nomes de dias e meses para Português
    
    mapa_dia_semana = F.create_map(
        F.lit("Sun"), F.lit("Domingo"),
        F.lit("Mon"), F.lit("Segunda-feira"),
        F.lit("Tue"), F.lit("Terça-feira"),
        F.lit("Wed"), F.lit("Quarta-feira"),
        F.lit("Thu"), F.lit("Quinta-feira"),
        F.lit("Fri"), F.lit("Sexta-feira"),
        F.lit("Sat"), F.lit("Sábado")
    )
    
    mapa_mes = F.create_map(
        F.lit("January"), F.lit("Janeiro"),
        F.lit("February"), F.lit("Fevereiro"),
        F.lit("March"), F.lit("Março"),
        F.lit("April"), F.lit("Abril"),
        F.lit("May"), F.lit("Maio"),
        F.lit("June"), F.lit("Junho"),
        F.lit("July"), F.lit("Julho"),
        F.lit("August"), F.lit("Agosto"),
        F.lit("September"), F.lit("Setembro"),
        F.lit("October"), F.lit("Outubro"),
        F.lit("November"), F.lit("Novembro"),
        F.lit("December"), F.lit("Dezembro")
    )
    
    df_dim_tempo_pt = df_dim_tempo.withColumn(
        "dia_da_semana_nome", mapa_dia_semana[F.col("dia_da_semana_nome_en")]
    ).withColumn(
        "mes_nome", mapa_mes[F.col("mes_nome_en")]
    ).withColumn(
        "eh_fim_de_semana",
        F.when(F.col("dia_da_semana_num").isin(1, 7), "Sim").otherwise("Não")
    )
    
    # 5. Selecionar colunas finais (conforme tabela do PDF)
    df_final = df_dim_tempo_pt.select(
        "sk_tempo",
        "ano",
        "trimestre",
        "mes",
        "semana_do_ano",
        "dia",
        "dia_da_semana_num",
        "dia_da_semana_nome",
        "mes_nome",
        "eh_fim_de_semana"
    )

    df_final.write.mode("overwrite").saveAsTable("medalhao.gold.dm_tempo")
    
    print("dimensão medalhao.gold.dm_tempo criada.")

except Exception as e:
    print(f"erro em 3.1: {e}")

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import DecimalType

try:
    # 1. Carregar tabelas Silver
    df_pedidos = spark.table("medalhao.silver.ft_pedidos")
    df_itens = spark.table("medalhao.silver.ft_itens_pedidos")
    df_cotacao = spark.table("medalhao.silver.dm_cotacao_dolar")
    
    # 2. Carregar e Limpar a tabela de avaliações
    df_avaliacoes_raw = spark.table("medalhao.silver.ft_avaliacoes_pedidos")

    # Usar try_cast (via F.expr) para converter a coluna 'avaliacao' para número. Textos corrompidos virarão NULL.
    df_avaliacoes_clean = df_avaliacoes_raw.withColumn(
        "avaliacao_num",
        F.expr("try_cast(avaliacao AS double)")
    )
    
    # 3. Pré-processar: Calcular a média da avaliação por pedido
    # Agora usamos a nova coluna limpa 'avaliacao_num'
    df_avg_avaliacoes = df_avaliacoes_clean.groupBy("id_pedido") \
                                     .agg(F.avg("avaliacao_num").alias("avaliacao_pedido"))

    # 4. Preparar a tabela de pedidos para o join com cotação
    df_pedidos_com_data = df_pedidos.withColumn("fk_tempo", F.to_date(F.col("pedido_compra_timestamp")))

    # 5. Juntar (join) todas as fontes de dados
    df_joined = df_itens.join(
        df_pedidos_com_data,
        "id_pedido",
        "left"
    ).join(
        df_cotacao,
        df_pedidos_com_data["fk_tempo"] == df_cotacao["data"],
        "left"
    ).join(
        df_avg_avaliacoes,
        "id_pedido",
        "left"
    )

    # 6. Calcular colunas de valor
    df_final = df_joined.withColumn(
        "valor_total_item_brl", (F.col("preco_BRL") + F.col("preco_frete"))
    ).withColumn(
        "valor_produto_usd", (F.col("preco_BRL") / F.col("cotacao_dolar"))
    ).withColumn(
        "valor_frete_usd", (F.col("preco_frete") / F.col("cotacao_dolar"))
    ).withColumn(
        "valor_total_item_usd", (F.col("valor_total_item_brl") / F.col("cotacao_dolar"))
    )

    # 7. Selecionar e renomear colunas finais (conforme tabela do PDF)
    df_gold = df_final.select(
        F.col("id_pedido"),
        F.col("id_item"),
        F.col("id_consumidor").alias("fk_cliente"),
        F.col("id_produto").alias("fk_produto"),
        F.col("id_vendedor").alias("fk_vendedor"),
        F.col("fk_tempo"),
        F.col("status").alias("status_pedido"),
        F.col("tempo_entrega_dias"),
        F.col("entrega_no_prazo"),
        
        # Valores BRL
        F.col("preco_BRL").cast(DecimalType(12, 2)).alias("valor_produto_brl"),
        F.col("preco_frete").cast(DecimalType(12, 2)).alias("valor_frete_brl"),
        F.col("valor_total_item_brl").cast(DecimalType(12, 2)).alias("valor_total_item_brl"),
        
        # Valores USD
        F.col("valor_produto_usd").cast(DecimalType(12, 2)).alias("valor_produto_usd"),
        F.col("valor_frete_usd").cast(DecimalType(12, 2)).alias("valor_frete_usd"),
        F.col("valor_total_item_usd").cast(DecimalType(12, 2)).alias("valor_total_item_usd"),
        
        F.col("cotacao_dolar").cast(DecimalType(8, 4)).alias("cotacao_dolar"),
        F.col("avaliacao_pedido").cast(DecimalType(3, 2)).alias("avaliacao_pedido")
    )

    # 8. Salvar na camada Gold
    df_gold.write.mode("overwrite").saveAsTable("medalhao.gold.ft_vendas_geral")
    
    print("tabela medalhao.gold.ft_vendas_geral criada.")

except Exception as e:
    print(f"erro em 3.2: {e}")

In [0]:
from pyspark.sql import functions as F

# Query SQL para criar a view (com a coluna 'dia_da_semana_nome' adicionada)
query_view_3_3 = """
    CREATE OR REPLACE VIEW medalhao.gold.view_vendas_por_periodo AS
    SELECT 
        -- Dimensões de Tempo
        t.ano,
        t.trimestre,
        t.mes,
        t.mes_nome,
        t.dia,
        t.dia_da_semana_num,
        t.dia_da_semana_nome, -- <<< CORREÇÃO: Coluna adicionada
        
        -- Indicadores (KPIs)
        COUNT(DISTINCT v.id_pedido) AS total_pedidos,
        COUNT(v.id_item) AS total_itens,
        SUM(v.valor_total_item_brl) AS receita_total_brl,
        SUM(v.valor_total_item_usd) AS receita_total_usd,
        AVG(v.valor_total_item_brl) AS ticket_medio_brl,
        AVG(v.avaliacao_pedido) AS avaliacao_media
        
    FROM medalhao.gold.ft_vendas_geral AS v
    
    -- Juntar com a dimensão de tempo
    LEFT JOIN medalhao.gold.dm_tempo AS t
        ON v.fk_tempo = t.sk_tempo
        
    GROUP BY 
        t.ano,
        t.trimestre,
        t.mes,
        t.mes_nome,
        t.dia,
        t.dia_da_semana_num,
        t.dia_da_semana_nome -- <<< CORREÇÃO: Coluna adicionada
"""

try:
    spark.sql(query_view_3_3)
    print("view medalhao.gold.view_vendas_por_periodo criada.")
except Exception as e:
    print(f"erro em 3.3: {e}")

In [0]:

print("Pergunta 1: Qual é o dia da semana com maior receita total em reais?")
spark.sql("""
    SELECT 
        dia_da_semana_nome,
        SUM(receita_total_brl) AS total_faturado
    FROM medalhao.gold.view_vendas_por_periodo
    GROUP BY dia_da_semana_nome
    ORDER BY total_faturado DESC
    LIMIT 1
""").show()


print("\nPergunta 2: No último ano, qual mês teve o maior ticket médio?")
# Primeiro, encontramos o último ano disponível
ultimo_ano = spark.table("medalhao.gold.dm_tempo").selectExpr("MAX(ano)").first()[0]
print(f"(Analisando o último ano: {ultimo_ano})")

spark.sql(f"""
    SELECT 
        mes_nome,
        AVG(ticket_medio_brl) AS media_ticket_medio
    FROM medalhao.gold.view_vendas_por_periodo
    WHERE ano = {ultimo_ano}
    GROUP BY mes, mes_nome
    ORDER BY media_ticket_medio DESC
    LIMIT 1
""").show()

In [0]:
from pyspark.sql import functions as F

# Query SQL para criar a view analítica
# Esta view junta a fato de vendas com os dados dos produtos
query_view_3_4 = """
    CREATE OR REPLACE VIEW medalhao.gold.view_top_produto AS
    SELECT 
        v.fk_produto AS id_produto,
        p.categoria_produto,
        
        -- KPIs de Vendas
        COUNT(v.id_item) AS quantidade_vendida,
        COUNT(DISTINCT v.id_pedido) AS total_pedidos,
        
        -- KPIs de Receita
        SUM(v.valor_total_item_brl) AS receita_brl,
        SUM(v.valor_total_item_usd) AS receita_usd,
        AVG(v.valor_total_item_brl) AS preco_medio_brl,
        
        -- KPIs de Avaliação e Logística
        AVG(v.avaliacao_pedido) AS avaliacao_media,
        AVG(p.peso_produto_gramas) AS peso_medio_gramas
        
    FROM medalhao.gold.ft_vendas_geral AS v
    
    -- Juntar com a tabela de produtos para buscar a categoria e peso
    LEFT JOIN medalhao.silver.ft_produtos AS p
        ON v.fk_produto = p.id_produto
        
    GROUP BY
        v.fk_produto,
        p.categoria_produto
"""

try:
    spark.sql(query_view_3_4)
    print("view medalhao.gold.view_top_produto criada.")
except Exception as e:
    print(f"erro em 3.4: {e}")

In [0]:
from pyspark.sql import functions as F

# Query SQL para criar a view (usando CTE)
query_view_3_5 = """
    CREATE OR REPLACE VIEW medalhao.gold.view_vendas_produtos_esteticos AS
    
    -- Início da CTE: Selecionar e filtrar os dados base
    WITH VendasFashion AS (
        SELECT 
            t.ano,
            t.mes,
            p.categoria_produto,
            v.id_pedido,
            v.id_item,
            v.valor_total_item_brl,
            v.valor_total_item_usd,
            v.avaliacao_pedido
        FROM medalhao.gold.ft_vendas_geral AS v
        
        -- Join para buscar categoria
        LEFT JOIN medalhao.silver.ft_produtos AS p
            ON v.fk_produto = p.id_produto
            
        -- Join para buscar ano/mês
        LEFT JOIN medalhao.gold.dm_tempo AS t
            ON v.fk_tempo = t.sk_tempo
            
        -- Filtro da categoria "Fashion"
        WHERE p.categoria_produto LIKE 'fashion%'
    )
    
    -- Query principal: Agregar os dados da CTE
    SELECT
        ano,
        mes,
        categoria_produto,
        
        -- KPIs
        COUNT(DISTINCT id_pedido) AS total_pedidos,
        COUNT(id_item) AS total_itens_vendidos,
        SUM(valor_total_item_brl) AS receita_total_brl,
        SUM(valor_total_item_usd) AS receita_total_usd,
        AVG(valor_total_item_brl) AS ticket_medio_brl,
        AVG(valor_total_item_usd) AS ticket_medio_usd,
        AVG(avaliacao_pedido) AS avaliacao_media
        
    FROM VendasFashion
    GROUP BY
        ano,
        mes,
        categoria_produto
"""

try:
    spark.sql(query_view_3_5)
    print("view medalhao.gold.view_vendas_produtos_esteticos criada.")
except Exception as e:
    print(f"erro em 3.5: {e}")